In [1]:
!pip install torch==2.5.1

In [5]:
!pip install bitsandbytes accelerate transformers

In [3]:
!pip install -r ../requirements.txt

  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached flask-3.1.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached Flask_SocketIO-5.5.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached loguru-0.7.3-py3-none-any.whl.metadata (22 kB)
  Using cached lxml-5.4.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.5 kB)
  Using cached pandas-2.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached pdfplumber-0.11.6-py3-none-any.whl.metadata (42 kB)
  Using cached pydantic-2.11.5-py3-none-any.whl.metadata (67 kB)
  Using cached uvicorn-0.34.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached starlette-0.46.2-py3-none-any.whl.metadata (6.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached blinker-1.9.0-py

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 1. 모델 및 토크나이저 로드
model_name = "dnotitia/Llama-DNA-1.0-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                # 4-bit 양자화
    bnb_4bit_use_double_quant=True,   # 2단계 양자화 사용 (효율성 향상)
    bnb_4bit_quant_type="nf4",        # 양자화 타입 (nf4 추천)
    bnb_4bit_compute_dtype=torch.bfloat16  # 연산 dtype (bf16 또는 float16)
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,   # 양자화 config 전달
    device_map="auto",
    resume_download=True
)


categories = {
    0: "졸업요건",
    1: "학교 공지사항",
    2: "학사일정",
    3: "식단 안내",
    4: "통학/셔틀 버스"
}


import random

def classify_query(user_query):
    """
    사용자 질의를 입력받아 5가지 카테고리 중 하나로 분류합니다.
    실패 시 무작위(0~4)로 fallback하여 절대 실패하지 않습니다.
    """

    prompt = f"""다음은 사용자 질문과 관련된 카테고리 목록입니다:
0: 졸업요건 (예: 졸업까지 몇 학점을 들어야 하나요?)
1: 학교 공지사항 (예: 이번에 올라온 공지사항 어디서 볼 수 있어요?)
2: 학사일정 (예: 이번 학기 수강신청은 언제 시작하나요?)
3: 식단 안내 (예: 오늘 학식 뭐 나와요?)
4: 통학/셔틀 버스 (예: 다음주에 셔틀버스는 정상 운행하나요?)

사용자 질문: "{user_query}"
위 질문은 어떤 카테고리에 가장 적합한가요? 숫자 레이블만 대답해주세요 (0, 1, 2, 3, 4 중 하나).

가장 적합한 카테고리 번호: """


    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=5,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,
        temperature=0.5
    )

    response_text = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()

    try:
        predicted_label_str = ''.join(filter(str.isdigit, response_text))
        if predicted_label_str:
            predicted_label = int(predicted_label_str[0])
            if predicted_label in categories:
                return predicted_label, categories[predicted_label]
    except Exception:
        pass

    category_examples = {
        0: "졸업 요건, 졸업 학점, 졸업 논문, 졸업 자격, 졸업 인증, 유예 신청, 졸업 필수 조건",
        1: "학교 공지사항, 안내문, 공고, 공지 업데이트, 휴강 안내, 장학금 공지, 긴급 알림",
        2: "수강 신청, 시험 기간, 성적 발표, 개강일, 종강일, 학사 일정, 수업 일정, 등록 일정",
        3: "학식, 학생 식당, 식단표, 조식, 중식, 석식, 오늘 메뉴, 급식 시간, 식단 운영",
        4: "셔틀버스, 통학버스, 운행 시간표, 버스 노선, 정류장 위치, 셔틀 예약, 통학 교통"
    }


    best_score = 0
    best_label = None
    for label, example in category_examples.items():
        score = difflib.SequenceMatcher(None, user_query, example).ratio()
        if score > best_score:
            best_score = score
            best_label = label

    return best_label, categories[best_label]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
import json
from pathlib import Path
from tqdm import tqdm

# data/question directory relative to this notebook
base_dir = Path('../data/question') if Path('../data/question').exists() else Path('data/question')
output_dir = Path('../outputs') if Path('../outputs').exists() else Path('outputs')
output_dir.mkdir(parents=True, exist_ok=True)

for path in base_dir.glob('*.json'):
    with path.open('r', encoding='utf-8') as f:
        questions_data = json.load(f)
    results = []
    for item in tqdm(questions_data, desc=path.name):
        question = item['question']
        label, _ = classify_query(question)  # VARCO 모델 사용
        results.append({'question': question, 'label': label})
    out_file = output_dir / f"{path.stem}_output.json"
    with out_file.open('w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    print(f'✅ {out_file} 저장 완료')


randomized_korean_questions_result.json: 100%|████████████████████████████| 104/104 [00:29<00:00,  3.58it/s]


✅ ../outputs/randomized_korean_questions_result_output.json 저장 완료


classifier_questions_only_result.json: 100%|██████████████████████████████| 147/147 [00:41<00:00,  3.55it/s]

✅ ../outputs/classifier_questions_only_result_output.json 저장 완료
